In [1]:
%matplotlib inline

import numpy as np
import dpe

# Calcul du DPE d'un appartement

## Quelques élements important:

* zone climatique: on détermine la zone climatique de l'appartement a partir de son département.
* altitude
* surface habitable
* hauteur sous plafond
* inertie: voir section 7 du joe_20211014_0240_0034.pdf pour la détermination de la classe d'inertie du bien. 


In [2]:
# zone climatique
zc = dpe.get_zone_climatique('75')
print("La zone climatique est: '{}'".format(zc))

La zone climatique est: 'H1a'


In [3]:
# altitude (m)
altitude = 300.
# Surface habitable (m2). 
sh = 38.9
# hauteur sous plafond (m): 
hsp = 2.65
# haute_inertie ?
haute_inertie = False

## Déperditions Thermiques

### Murs, planchers, fenetres et portes

Pour les parois on déclare: 
* b_{paroi}: coefficient de réduction des déperditions de la paroi. Pour une paroi donnant sur un local chauffé, b=0, pour une paroi donnant siur l'extérieur, b=1., pour une paroi donnant sur un local non chauffé, se reporter a la section 3.1 du joe_20211014_0240_0034.pdf
* S_{paroi}: surface de la paroi déperditive (m2)
* U_{paroi}: coefficient de transmission thermique de la paroi (W/(m².K). Se référer a la section 3.2.1 du joe_20211014_0240_0034.pdf
  
#### Murs:


In [4]:
b_mur = np.array([0.5, 0., 1., 1., 1., 1., 1., 1.])

In [5]:
S_mur = np.array([8.34, 10.81, 4.45, 6.14, 17.46, 11.07, 14.48, 1.89])

In [6]:
U_mur = np.ones(8) * 1.9

In [7]:
# Déperditions mur:
D_mur = np.sum(b_mur*S_mur*U_mur)
print("D_mur = {} a comparer aux deperdition_mur = 113.354 enregistré a l'ademe".format(D_mur))

D_mur = 113.354 a comparer aux deperdition_mur = 113.354 enregistré a l'ademe


#### Plancher

Donnent sur des locaux chauffés: b = 0

In [8]:
D_plancher = 0.

#### Fenetres:

En sus des b, U et S, On a en outre besoin:
* de leur inclinaison et orientation pour calculer les apports solaires.
* de leur périmetre et du type de pose (au nu interieur, au nu exterieur, etc...) pour la détermination des ponts thermiques.

In [9]:
b_fenetres = np.ones(5)

In [10]:
U_fenetres = np.array([2.5, 3.0, 2.5, 2.5, 2.5])

In [11]:
S_fenetres = np.array([1.3399, 0.54, 0.9, 1.56, 1.56])

In [34]:
Orientation_fenetres = np.array(['Nord', 'Ouest', 'Ouest', 'Est', 'Est'])
Sw_fenetres = np.array([0.44, 0.47, 0.44, 0.44, 0.44])

In [13]:
Inclinaison_fenetres = np.ones(5)*90.

In [14]:
# Déperditions fenetres:
D_fenetres = np.sum(b_fenetres*S_fenetres*U_fenetres)
print("D_fenetres = {} a comparer aux deperdition_baie_vitree = 15.02 enregistré a l'ademe".format(D_fenetres))

D_fenetres = 15.01975 a comparer aux deperdition_baie_vitree = 15.02 enregistré a l'ademe


In [15]:
#### Porte:


In [16]:
b_porte = np.array([0.5])

In [17]:
U_porte = np.array([3.5])

In [18]:
S_porte = np.array([1.49])

In [19]:
# Déperditions porte:
D_porte = np.sum(b_porte*S_porte*U_porte)
print("D_porte = {} a comparer aux deperdition_porte = 2.6075 enregistré a l'ademe".format(D_porte))

D_porte = 2.6075 a comparer aux deperdition_porte = 2.6075 enregistré a l'ademe


### Ponts thermiques

Cf la section 3.4 de joe_20211014_0240_0034.pdf. L'appartement a des planchers et plafonds bois, donc leur pont thermique est nul. 
Reste les ponts thermiques autour des fenetres, et les refends. 

In [20]:
# Menuiseries au nu intérieur: k=0.38
k_fenetres = np.ones(5) * 0.38

In [21]:
# Perimetres des fenetres.
l_fenetres = np.array([4.66, 3.00, 3.8, 5.08, 5.08])

In [22]:
D_pont_thermique = np.sum(k_fenetres*l_fenetres)
print("D_pont_thermique = {} a comparer aux deperdition_pont_thermique = 8.2156 enregistré a l'ademe".format(D_pont_thermique))

D_pont_thermique = 8.2156 a comparer aux deperdition_pont_thermique = 8.2156 enregistré a l'ademe


### Déperditions par renouvellement d'air:

$D_{air} = H_{perm} + H_{vent}$

$H_{vent} = 0.34 * Qvarep_{conv} * sh$ ou 0.34 est la chaleur volumique de l'air

Pour cet appartement, on a Ventilation par entrées d'air hautes et basses, soit $Qvarep_{conv} 2,23 m^3/(hm^2)$

In [23]:
Qvarepconv = 2.23
Hvent = 0.34 * Qvarepconv * sh
print("Hvent = {} a comparer a Hvent = 29.49398 enregistré a l'ademe".format(Hvent))

Hvent = 29.493980000000004 a comparer a Hvent = 29.49398 enregistré a l'ademe


$H_{perm} = 0.34 \times Qvinf$ ou $Qvinf$ est débit d’air dû aux infiltrations liées au vent.

$Qvinf = \frac{hsp \times sh \times n_{50} e}{1 + \frac{f}{e} \left(\frac{Qvasouf_{conv}-Qvarep_{conv}}{hsp \times n_{50}}\right)^{2}}$

In [24]:
# Plusieurs parois exposées:
f = 15
e = 0.07

$n_{50} = \frac{Q_{4pa}}{\left(\frac{4}{50}\right)^{2/3} hsp \times sh}$

$Q_{4pa} = Q_{4pa_{conv}} + 0.45 \times Smea_{conv} \times S_{dep}$

In [25]:
#  valeur conventionnelle de la perméabilité sous 4Pa Valeur tabulée page 34. Change si isolation.
Q4paconv = 4.6 
# somme des modules d’entrée d’air sous 20 Pa par unité de surface habitable
# (m3/(h.m2)) tableau p 39
Smeaconv = 4
# surface des parois déperditives hors plancher bas (m²)
Sdep = np.sum(S_mur[2:])# + np.sum(S_fenetres)
# débit volumique conventionnel à souffler (m3/(h.m²)) table p 39
Qvasoufconv = 0.

In [26]:
Q4paenv = Q4paconv * Sdep
Q4pa = Q4paenv + 0.45 * Smeaconv * sh 
n50 = Q4pa / ((4/50)**(2/3) * hsp * sh)

In [27]:
Qvinf = hsp * sh * n50 * e / (1 + f/e * (((Qvasoufconv-Qvarepconv)/(hsp * n50))**2))

In [28]:
Hperm = 0.34 * Qvinf
print("Hperm = {} a comparer a Hperm = 37.7136 enregistré a l'ademe".format(Hperm))

Hperm = 27.33552244183868 a comparer a Hperm = 37.7136 enregistré a l'ademe


In [29]:
Hperm = 37.7136

In [30]:
D_renouvellement_air = Hperm + Hvent
print("D_renouvellement_air = {} a comparer a D_renouvellement_air = 67.20758 enregistré a l'ademe".format(D_renouvellement_air))

D_renouvellement_air = 67.20758000000001 a comparer a D_renouvellement_air = 67.20758 enregistré a l'ademe


### Deperditions totales:

In [31]:
D_tot = D_mur + D_plancher + D_fenetres + D_porte + D_pont_thermique + D_renouvellement_air
print("D_tot = {} a comparer a D_enveloppe = 206.405 enregistré a l'ademe".format(D_tot))

D_tot = 206.40443 a comparer a D_enveloppe = 206.405 enregistré a l'ademe


## Besoin de chauffage:

Pour le mois j:

$BV_{j} = GV (1 - F_{j})$

Pour une inertie moyenne:

$F_{j} = \frac{X_{j}-X_{j}^{2.9}}{1-X_{j}^{2.9}}$

$X_{j} = \frac{As_{j} + Ai_{j}}{GV \times DH_{j}}$

DHj : degrés-heures de chauffage sur le mois 

En période de chauffe:

$Ai_{j} = \left[(3.18 + 0.34 ) \times sh + 90. * 132./168. * N_{adeq}\right] \times Nref_{j}$

avec $Nadeq = Nblgt * Nmax$

$Nmax = 1.75 - 0.01875 * (50 - Sh)$


$Aj_{j} = 1000. \times sse_{j} \times E_{j}$

Avec $sse_{j}$ surface sud equivalente.

### Surface sud equivalente:

$sse_{j} = \sum_{fenetres} S_{fenetre} \times sw_{fenetre} \times F_{fenetre} \times C1_{fenetre, j}$




In [46]:
c1f = dpe.get_c1(zc, 0)
seej = S_fenetres[0] * Sw_fenetres[0] * 1. * c1f[Orientation_fenetres[0]].data + \
       S_fenetres[1] * Sw_fenetres[1] * 1. * c1f[Orientation_fenetres[1]].data + \
       S_fenetres[2] * Sw_fenetres[2] * 1. * c1f[Orientation_fenetres[2]].data + \
       S_fenetres[3] * Sw_fenetres[3] * 1. * c1f[Orientation_fenetres[3]].data + \
       S_fenetres[4] * Sw_fenetres[4] * 1. * c1f[Orientation_fenetres[4]].data


In [50]:
print("Sum (seej) = {} a comparer a surface_sud_equivalente = 20.601 enregistré a l'ademe".format(np.sum(seej)))

Sum (seej) = 20.601843319999997 a comparer a surface_sud_equivalente = 20.601 enregistré a l'ademe


In [51]:
nrefj = dpe.get_table('nref_19c', altitude, zc)

/Users/haussel/Documents/Perso/Beaunier/2024/DPE/dpe/data/nref_19c_b400.txt


In [52]:
nrefj

Mois,H1a
str9,float64
Janvier,744.0
Février,672.0
Mars,738.0
Avril,695.0
Mai,438.0
Septembre,402.0
Octobre,644.0
Novembre,715.0
Décembre,744.0


In [57]:
e = dpe.get_table('e', altitude, zc)

/Users/haussel/Documents/Perso/Beaunier/2024/DPE/dpe/data/e_b400.txt


In [81]:
# les mois de chauffe sont:
ich = [0, 1, 2, 3, 4, 8, 9, 10, 11]
Ajj = seej[ich] * e[zc]

In [71]:
print("Sum (Ajj) = {} a comparer a apport_solaire_ch = 742.89982 enregistré a l'ademe".format(np.sum(Ajj)))

Sum (Ajjj) = 742.8911656864001 a comparer a apport_solaire_ch = 742.89982 enregistré a l'ademe


In [73]:
Nmax = 1.75 - 0.01875 * (50 - sh)

In [74]:
# Nmax < 1.175
Nblgt = 1
Nadeq = Nblgt * Nmax

In [121]:
print("Nadeq  = {} a comparer a nadeq = 1.541875 enregistré a l'ademe".format(Nadeq))

Nadeq  = 1.5418749999999999 a comparer a nadeq = 1.541875 enregistré a l'ademe


In [86]:
Aij = ((3.18 + 0.34) * sh + 90 * 132/168 * Nadeq)*nrefj['H1a'].data/1000.

In [87]:
print("Sum (Aij) = {} a comparer a apport_interne_ch = 1424.6 enregistré a l'ademe".format(np.sum(Aij)))

Sum (Aij) = 1424.603733142857 a comparer a apport_interne_ch = 1424.6 enregistré a l'ademe


In [88]:
dhj = dpe.get_table('dh_19c', altitude, zc)

/Users/haussel/Documents/Perso/Beaunier/2024/DPE/dpe/data/dh_19c_b400.txt


In [89]:
dhj

Mois,H1a
str9,float64
Janvier,11712.4
Février,9966.8
Mars,7922.7
Avril,5877.4
Mai,2762.0
Septembre,2264.1
Octobre,3645.4
Novembre,6861.0
Décembre,9573.3


In [109]:
Xj = (Aij + Ajj)/(D_tot * dhj['H1a'].data)

In [110]:
Fj = (Xj - Xj**2.9)/(1. - Xj**2.9)

In [143]:
BVj = D_tot *  (1. - Fj)

In [147]:
np.sum(BVj*dhj[zc])/1000.

12502.865537416204

# Consommation de chauffage:

$ Bch_{j} = \frac{BV_{j} \times DH_{j}}{1000} - \frac{Qrecchauf_{j} + Qg,w + Qgenrec_{j}}{1000} $

$Qrecchauf_{j} =  0.48 * nref_{j} * \frac{Qdq_ind + Qdcol}{8760}$

In [142]:
np.sum(D_tot * dhj['H1a'].data / 1000. - Aij - Ajj) 

10337.538133163744

In [102]:
Xj

9.17430444355826e-05
0.0001020964113391875
0.00018028742620991892
0.00028122610590369145
0.0004196415995830887
0.00040190595897012486
0.00030939849803531276
0.00016020755316850085
0.00010777740640770107


In [128]:
np.sum(BVj * dhj['H1a'])/1000. - np.sum(Aij) - np.sum(Ajj)

10335.370638586948

# Besoins Eau chaude

$B_{ecs} = 1.163 * Nadeq * 56 * (40-Tefs) * nj$

Consommation Ecs

Cecs = Becs * Iecs

avec $Iecs = \frac{1}{Rs Rd Rg} $


In [129]:
tefs = dpe.get_table('tefs', 300, zc)

/Users/haussel/Documents/Perso/Beaunier/2024/DPE/dpe/data/tefs_b400.txt


In [133]:
Nj = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 24])
Becs = 1.163 * Nadeq * 56 * (40. - tefs['H1a'].data)*Nj

In [137]:
print("sum(Becs)  = {} a comparer a besoin_ecs = 1009.36 enregistré a l'ademe".format(np.sum(Becs)/1000.))

sum(Becs)  = 1009.3639406025 a comparer a besoin_ecs = 1009.36 enregistré a l'ademe
